In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install -U \
    transformers==4.27.2 \
    datasets --quiet

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [ ]:
#huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset("knkarthick/dialogsum")

In [ ]:
example_indices = [10, 102]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Exemplo ', i + 1)
    print(dash_line)
    print('DIÁLOGO DE ENTRADA:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('RESUMO HUMANO:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

In [ ]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

2

In [ ]:
sentence = "A Ana Cunha desenha melhor do que o Palla!"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0], 
        skip_special_tokens=True
    )

print('SENTENÇA CODIFICADA:')
print(sentence_encoded["input_ids"][0])
print('\nSENTENÇA DECODIFICADA:')
print(sentence_decoded)

# Exemplo 1 - Sem nenhuma engenharia de prompt.

In [13]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Exemplo ', i + 1)
    print(dash_line)
    print(f'PROMPT DE ENTRADA:\n{dialogue}')
    print(dash_line)
    print(f'RESUMO HUMANO:\n{summary}')
    print(dash_line)
    print(f'GERADO PELO MODELO - SEM ENGENHARIA DE PROMPT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Exemplo  1
---------------------------------------------------------------------------------------------------
PROMPT DE ENTRADA:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
RE

# Exemplo 2 - Engenharia de prompt zero shot com uma instrução.

In [14]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Exemplo ', i + 1)
    print(dash_line)
    print(f'PROMPT DE ENTRADA:\n{prompt}')
    print(dash_line)
    print(f'RESUMO HUMANO:\n{summary}')
    print(dash_line)    
    print(f'GERADO PELO MODELO - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Exemplo  1
---------------------------------------------------------------------------------------------------
PROMPT DE ENTRADA:

Summarize the following conversation.

#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

Summary:
    
-----------------------------------------------

# Exemplo 3 - Inferencia Zero Shot com modelo de prompt do FLAN-T5 ( Template de prompt)
https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py

In [15]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
        
    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )

    print(dash_line)
    print('Exemplo ', i + 1)
    print(dash_line)
    print(f'PROMPT DE ENTRADA:\n{prompt}')
    print(dash_line)
    print(f'RESUMO HUMANO:\n{summary}\n')
    print(dash_line)
    print(f'GERADO PELO MODELO - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Exemplo  1
---------------------------------------------------------------------------------------------------
PROMPT DE ENTRADA:

Dialogue:

#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

What was going on?

---------------------------------------------------------------------

# Exemplo com one shot

## Definindo o prompt

In [16]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""
    
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    
    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
        
    return prompt

## Constriuindo o prompt

In [17]:
example_indices_full = [10]
example_index_to_summarize = 102

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

What was going on?
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.



Dialogue:

#Person1#: Well, I'll see you later, Mrs. Todd. My wife is waiting for me to take her shopping.
#Person2#: I understand. There's a lot to get done at weekends, especially when you two work and th

## Utilizando o prompt

In [18]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'RESUMO HUMANO:\n{summary}\n')
print(dash_line)
print(f'GERADO PELO MODELO - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
RESUMO HUMANO:
#Person1# greets Mrs. Todd and then they say goodbye to each other.

---------------------------------------------------------------------------------------------------
GERADO PELO MODELO - ONE SHOT:
Person1 is going to visit Mrs. Todd later. Jane is waiting for her to take her shopping.


# Exemplo few shots

## Construindo o prompt

In [19]:
example_indices_full = [10, 80, 120]
example_index_to_summarize = 102

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

What was going on?
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.



Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This

In [20]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'RESUMO HUMANO:\n{summary}\n')
print(dash_line)
print(f'GERADO PELO MODELO - FEW SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (851 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
RESUMO HUMANO:
#Person1# greets Mrs. Todd and then they say goodbye to each other.

---------------------------------------------------------------------------------------------------
GERADO PELO MODELO - FEW SHOT:
Jane and Jane are going shopping at the weekend.


# Ajustando parâmetros

In [23]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=4.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'GERADO PELO MODELO - FEW SHOT:\n{output}')
print(dash_line)
print(f'RESUMO HUMANO:\n{summary}\n')

---------------------------------------------------------------------------------------------------
GERADO PELO MODELO - FEW SHOT:
Before talking, woman calls. Mrs
---------------------------------------------------------------------------------------------------
RESUMO HUMANO:
#Person1# greets Mrs. Todd and then they say goodbye to each other.

